In [2]:
from nltk.corpus import wordnet as wn
import re
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')
import json

/home/kjros2/.local/lib/python3.8/site-packages/huggingface_hub/file_download.py:588: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


In [3]:

#TODO missing optimal expansion to bm25s

# corpus disk12
# corpus covid

# queries 51-100
# queries round1

# source wordnet
# source optimal

# weight wordnet_wup
# weight wordnet_path
# weight contextual_mean
# weight contextual_term
# weight 0, 0.1, 0.5, 1
# weight norm_across_query


# method bm25 (use sanjeev's new changes, missing query terms in expansion given weight 1)
# method bm25s (requires assignment, cannot be done by optimal yet)


"""
(DONE) wordnet, bm25s
wordnet, bm25

(DONE) optimal, bm25s
optimal, bm25


All wordnet paths should work
optimal, 0/0.1/0.5/1, bm25
"""

#config = json.load(open("config_wordnet.json"))
config = json.load(open("config_optimal.json"))

# config has one of each above
for run in config:
    print(run)

    corpus = run["corpus"]
    queries = run["queries"]
    source = run["source"]
    weight = run["weight"]
    method = run["method"]

    #if source != "optimal": continue
    if weight != "norm_across_query": continue


    # load queries for regular trec
    if queries == "51-100":
        path_to_queries = "tools/topics-and-qrels/topics.adhoc.51-100.txt"
        all_query_words = {}
        with open(path_to_queries, "r") as f:
            current_id = None
            for line in f:
                #  '<num> Number:  051 \n',
                if "Number:" in line:
                    current_id = str(int(line.split()[2]))
                    all_query_words[current_id] = {}
                if "Topic:" in line:
                    query = line.split()[2:]
                    for query_word in query:
                        query_word = re.sub("[^A-Za-z]", "", query_word.lower())
                        if current_id is not None:
                            all_query_words[current_id][query_word] = {}
    elif queries == "round1":
        # load queries for covid
        path_to_queries = "src/main/resources/topics-and-qrels/topics.covid-round1.xml"
        all_query_words = {}
        with open(path_to_queries, "r") as f:
            for line in f:
                if "<topic number=" in line:
                    current_id = str(int(re.sub("[^0-9]", "", line)))
                    all_query_words[current_id] = {}
                if "<query>" in line:
                    line = re.sub("<query>", "", line)
                    line = re.sub("</query>", "", line)
                    for query_word in line.split():
                        query_word = re.sub("[^A-Za-z]", "", query_word.lower())
                        all_query_words[current_id][query_word] = {}


    if source == "wordnet":
        for query_id in all_query_words:
            for query_word in all_query_words[query_id]:
                synonyms = wn.synsets(query_word)
                if len(synonyms) == 0: continue
                orig_query_term_synset = synonyms[0]
                all_synonyms = list(set([synonym.name().split(".")[0] for synonym in synonyms]))
                all_synonyms = {x: 0 for x in all_synonyms if x != query_word}
                all_query_words[query_id][query_word] = all_synonyms

                

    elif source == "optimal":
        # now when they're loaded, they're in the correct format
        if corpus == "covid":
            all_query_words = json.load(open("expanded_queries/optimal_v3.covid.json", "r"))
        elif corpus == "disk12":
            all_query_words = json.load(open("expanded_queries/optimal_v3.51-100.json", "r"))


    if weight in ["wordnet_path", "wordnet_wup"]:
        for query_id in all_query_words:
            for query_word in all_query_words[query_id]:
                synonyms = wn.synsets(query_word)
                if len(synonyms) == 0: continue
                orig_query_term_synset = synonyms[0]
                term_synset_map = {syn.name().split(".")[0]: syn for syn in synonyms if syn.name().split(".")[0] != query_word}
                for expansion_term in all_query_words[query_id][query_word]:
                    if weight == "wordnet_path":
                        sim = orig_query_term_synset.path_similarity(term_synset_map[expansion_term])
                    elif weight == "wordnet_wup":
                        sim = orig_query_term_synset.path_similarity(term_synset_map[expansion_term])
                    all_query_words[query_id][query_word][expansion_term] = sim

    elif weight in ["contextual_mean", "contextual_term"]:
        for query_id in all_query_words:
            for query_word in all_query_words[query_id]:
                expansion_terms = list(all_query_words[query_id][query_word].keys())
                if expansion_terms == []: continue
                embeddings_exp = model.encode(expansion_terms, convert_to_tensor=True)
                if weight == "contextual_term":
                    embeddings_orig = model.encode([query_word], convert_to_tensor=True)
                elif weight == "contextual_mean":
                    embeddings_orig = model.encode([" ".join(list(all_query_words[query_id].keys()))], convert_to_tensor=True)
                cosine_scores = util.cos_sim(embeddings_orig, embeddings_exp)
                for i,exp_term in enumerate(expansion_terms):
                    all_query_words[query_id][query_word][exp_term] = max(0,float(cosine_scores[0][i]))


    elif weight in [0,0.1,0.5,1]:
        for query_id in all_query_words:
            for query_word in all_query_words[query_id]:
                for expansion_term in all_query_words[query_id][query_word]:
                    all_query_words[query_id][query_word][expansion_term] = weight

    elif weight == "norm_across_query":
        for query_id in all_query_words:
            total_added_weight = 0
            for query_word in all_query_words[query_id]:
                for expansion_term in all_query_words[query_id][query_word]:
                    total_added_weight += all_query_words[query_id][query_word][expansion_term]
            
            if total_added_weight == 0:
                continue
            else:
                for query_word in all_query_words[query_id]:
                    for expansion_term in all_query_words[query_id][query_word]:
                        all_query_words[query_id][query_word][expansion_term] = all_query_words[query_id][query_word][expansion_term] / total_added_weight
        
    elif weight == "norm_across_term":
        pass
    elif weight == "keep_optimal":
        pass

    out_path = "expanded_queries/" + corpus + "." + queries + "." + source + "." + str(weight) + "." + method + ".txt"
    with open(out_path, "w") as f:
        for query_id in all_query_words:
            for query_word in all_query_words[query_id]:
                if method == "bm25":
                    for expansion_term in all_query_words[query_id][query_word]:
                        f.write(query_id + "-" + expansion_term + " " + str(all_query_words[query_id][query_word][expansion_term]) + "\n")
                elif method == "bm25s":
                    expansion_output = ", ".join(["(" + expansion_term + ", " + str(all_query_words[query_id][query_word][expansion_term]) + ")" for expansion_term in all_query_words[query_id][query_word]])
                    if expansion_output == "": continue
                    f.write(query_id + "-" + query_word + " " + expansion_output + "\n")

{'corpus': 'disk12', 'queries': '51-100', 'source': 'optimal', 'weight': 0, 'method': 'bm25'}
{'corpus': 'disk12', 'queries': '51-100', 'source': 'optimal', 'weight': 0.1, 'method': 'bm25'}
{'corpus': 'disk12', 'queries': '51-100', 'source': 'optimal', 'weight': 0.5, 'method': 'bm25'}
{'corpus': 'disk12', 'queries': '51-100', 'source': 'optimal', 'weight': 1, 'method': 'bm25'}
{'corpus': 'disk12', 'queries': '51-100', 'source': 'optimal', 'weight': 'keep_optimal', 'method': 'bm25'}
{'corpus': 'disk12', 'queries': '51-100', 'source': 'optimal', 'weight': 'norm_across_query', 'method': 'bm25'}
{'corpus': 'covid', 'queries': 'round1', 'source': 'optimal', 'weight': 0, 'method': 'bm25'}
{'corpus': 'covid', 'queries': 'round1', 'source': 'optimal', 'weight': 0.1, 'method': 'bm25'}
{'corpus': 'covid', 'queries': 'round1', 'source': 'optimal', 'weight': 0.5, 'method': 'bm25'}
{'corpus': 'covid', 'queries': 'round1', 'source': 'optimal', 'weight': 1, 'method': 'bm25'}
{'corpus': 'covid', 'quer